Importação das Bibliotecas Necessárias

In [22]:
import os
import re
import time
import pandas as pd
import win32com.client
import tempfile

Função para Conectar ao SAP

In [23]:
def conectar_sap(timeout=30):
    print("Aguardando o SAP GUI estar disponível...")
    for i in range(timeout):
        try:
            SapGuiAuto = win32com.client.GetObject("SAPGUI")
            application = SapGuiAuto.GetScriptingEngine
            if application.Children.Count > 0:
                connection = application.Children(0)
                if connection.Children.Count > 0:
                    session = connection.Children(0)
                    print("Conexão com SAP estabelecida com sucesso!")
                    return session
        except Exception:
            pass
        time.sleep(1)
    print("Erro: SAP GUI não pôde ser conectado após 30 segundos.")
    return None


Função para Aguardar Elementos no SAP

In [24]:
def aguardar_elemento(session, elemento_id, timeout=30):
    for _ in range(timeout):
        try:
            session.findById(elemento_id)
            return True
        except:
            time.sleep(1)
    raise Exception(f"Elemento {elemento_id} não encontrado após {timeout} segundos.")


Funções do Script SAP - Parte 1 (Execução)

In [25]:
def executar_script_sap1(session, projeto):
    try:
        print(f"📝 Inserindo projeto: {projeto}")
        campo_projeto = "wnd[0]/usr/ctxtP_PSPID"
        
        if aguardar_elemento(session, campo_projeto):
            session.findById(campo_projeto).text = projeto
            session.findById(campo_projeto).caretPosition = len(projeto)
        else:
            raise Exception(f"❌ Não foi possível encontrar o campo de projeto: {campo_projeto}")

        print("➡️ Executando transação...")
        session.findById("wnd[0]/tbar[1]/btn[8]").press()

        print("📂 Expandindo estrutura do projeto...")
        session.findById("wnd[0]/tbar[1]/btn[35]").press()
        
        shell_id = "wnd[0]/shellcont/shell"

        # Verifica se a estrutura esperada está presente
        try:
            shell = session.findById(shell_id)
            shell.expandNode("          9")
            shell.selectedNode = "         10"
            shell.topNode = "          1"
        except Exception:
            raise Exception("⚠️ Layout inesperado ou estrutura do projeto não encontrada. Pulando projeto.")

        print("📤 Exportando relatório...")
        session.findById("wnd[0]/tbar[1]/btn[25]").press()  # Exportar
        session.findById("wnd[0]/tbar[1]/btn[32]").press()  # Mais opções
        session.findById("wnd[0]/tbar[1]/btn[28]").press()  # Excel

    except Exception as e:
        raise Exception(f"❌ Erro na execução da primeira parte do script para o projeto {projeto}: {e}")


Funções do Script SAP - Parte 2 (Finalização e Limpeza)

In [26]:
def executar_script_sap2(session, projeto):
    try:
        print(f"🧹 Finalizando exportação e limpando SAP para o projeto {projeto}...")

        # Maximiza a janela novamente (caso algo tenha minimizado)
        session.findById("wnd[0]").maximize()

        # Fecha a transação (botão "Voltar")
        session.findById("wnd[0]/tbar[0]/btn[15]").press()

        # Pressiona botão de confirmação "Sim", se a janela aparecer
        try:
            session.findById("wnd[1]/usr/btnBUTTON_1").press()
        except:
            print("⚠️ Nenhuma confirmação extra após exportação.")

        # Volta à tela inicial
        try:
            session.findById("wnd[0]/tbar[0]/btn[3]").press()
            session.findById("wnd[0]/usr/ctxtP_PSPID").text = ""
            session.findById("wnd[0]/usr/ctxtP_PSPID").caretPosition = 0
        except:
            print("⚠️ Botão de voltar à tela inicial não encontrado.")

        print("✅ SAP limpo e pronto para o próximo projeto.")

    except Exception as e:
        print(f"❌ Erro ao finalizar o projeto {projeto}: {e}")


Função para Limpar Nome de Arquivo

In [27]:
def limpar_nome_arquivo(nome):
    return re.sub(r'[<>:"/\\|?*]', '_', nome)


Função para Salvar o Excel como CSV

In [28]:
def salvar_excel_aberto_como_csv(nome_procurado, pasta_destino, nome_projeto=None):
    excel = win32com.client.Dispatch("Excel.Application")
    arquivos_convertidos = []

    for wb in excel.Workbooks:
        if nome_procurado.lower() in wb.Name.lower():
            try:
                nome_base = limpar_nome_arquivo(wb.Name)
                if nome_projeto:
                    nome_csv = f"{limpar_nome_arquivo(nome_projeto)}.csv"
                else:
                    nome_csv = os.path.splitext(nome_base)[0] + ".csv"

                caminho_csv = os.path.join(pasta_destino, nome_csv)

                if wb.Path:
                    caminho_original = wb.FullName
                else:
                    temp_path = os.path.join(tempfile.gettempdir(), nome_base + ".xlsx")
                    wb.SaveAs(temp_path)
                    caminho_original = temp_path

                tab_tratada = pd.read_excel(caminho_original, sheet_name=wb.ActiveSheet.Name)
                tab_tratada.to_csv(caminho_csv, index=False, encoding="utf-8")
                arquivos_convertidos.append(caminho_csv)

                if not wb.Path:
                    os.remove(caminho_original)

            except Exception as e:
                print(f"Erro ao converter {wb.Name}: {e}")

    return arquivos_convertidos


Função para Processar os Projetos

In [29]:
def processar_projetos():
    pasta_destino = r"C:\Users\BR0IEPB\Documents\Treinamento_F-LT"
    nome_arquivo_excel = "BOM"  # Nome esperado do Excel gerado pelo SAP

    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)

    # Carregar lista de projetos
    df = pd.read_excel("projetos_selecionados.xlsx")
    lista_projetos = df["WBS Element"].dropna().tolist()

    # Conectar ao SAP
    session = conectar_sap()

    if session:
        for projeto in lista_projetos:
            try:
                print(f"\nExecutando projeto: {projeto}")
                
                # Executar a primeira parte do script SAP
                executar_script_sap1(session, projeto)

                # Esperar o SAP concluir o processo e o Excel carregar o relatório
                time.sleep(40)

                # Salvar o relatório do Excel como CSV
                arquivos = salvar_excel_aberto_como_csv(nome_arquivo_excel, pasta_destino, nome_projeto=projeto)
                if arquivos:
                    print(f"✅ Arquivos salvos para o projeto {projeto}:")
                    for a in arquivos:
                        print(f"- {a}")
                else:
                    print(f"⚠️ Nenhum relatório encontrado para o projeto {projeto}.")

                # Executar a segunda parte do script SAP
                executar_script_sap2(session, projeto)

            except Exception as e:
                print(f"❌ Erro ao processar o projeto {projeto}: {e}")

        print("\n✅ Processo concluído.")
    else:
        print("❌ Não foi possível conectar ao SAP.")


Execução do Processamento dos Projetos

In [ ]:
processar_projetos()

AGRUPANDO CSV'S NUM ÚNICO ARQUIVO

In [31]:
pasta_csvs = r"C:\Users\BR0IEPB\Documents\Treinamento_F-LT"

# Caminho de saída
arquivo_saida = os.path.join(pasta_csvs, "agrupado.csv")

# Lista todos os CSVs da pasta
csvs = [f for f in os.listdir(pasta_csvs) if f.lower().endswith(".csv")]

# Abre o arquivo final para escrita
with open(arquivo_saida, "w", encoding="utf-8", newline="") as f_saida:
    for arquivo in csvs:
        caminho = os.path.join(pasta_csvs, arquivo)
        
        # Escreve o nome do arquivo como uma linha separadora
        f_saida.write(f"\n# ==== {arquivo} ====\n")
        
        # Lê e escreve o conteúdo do CSV
        with open(caminho, "r", encoding="utf-8") as f_origem:
            for linha in f_origem:
                f_saida.write(linha)

print(f"✅ Arquivo final criado: {arquivo_saida}")

✅ Arquivo final criado: C:\Users\BR0IEPB\Documents\Treinamento_F-LT\agrupado.csv


In [ ]:
import pandas as pd
import os

# === CONFIGURAÇÃO ===
# Caminho da pasta onde estão os CSVs de origem
pasta_origem = r'C:\Users\BR0IEPB\Documents\Treinamento_F-LT\CSV'  # <- Altere aqui
# Caminho onde o arquivo final será salvo
pasta_destino = r'C:\Users\BR0IEPB\Documents\Treinamento_F-LT\CSV'  # <- Altere aqui
# Nome do arquivo final
nome_arquivo_saida = 'agrupado.csv'

# === PROCESSAMENTO ===
# Lista para armazenar os DataFrames
lista_df = []

# Garante que o diretório de destino exista
os.makedirs(pasta_destino, exist_ok=True)

# Percorre todos os arquivos CSV na pasta de origem
for nome_arquivo in os.listdir(pasta_origem):
    if nome_arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(pasta_origem, nome_arquivo)
        nome_base = os.path.splitext(nome_arquivo)[0]

        print(f'Processando: {nome_arquivo}')
        try:
            df = pd.read_csv(caminho_arquivo)
            df.insert(0, 'Projeto', nome_base)
            lista_df.append(df)
        except Exception as e:
            print(f'Erro ao processar {nome_arquivo}: {e}')

# Junta todos os DataFrames
df_final = pd.concat(lista_df, ignore_index=True)

# Caminho completo do arquivo de saída
caminho_saida = os.path.join(pasta_destino, nome_arquivo_saida)

# Salva o arquivo agrupado
df_final.to_csv(caminho_saida, index=False, encoding='utf-8')

print(f'\n✅ Arquivo final salvo em: {caminho_saida}')
